# Analysis by Google Trends' Topic Subsamples
This notebook cuts our dataset into equal-sized sub-samples based on the Google trends' topic (business, entertainment, health, sports, science, top stories) to see how our data varies based on these topics.

This notebook really only works using the full dataset `subsample=False`, as each topic subsample is 700 records. 


Some code we ran to get here includes some top-level analysis of trending searches and the creation of a dataset that contains unqiue search queries per topic (`fn_trends`). 

We also found that on average search queries were two words and 15 characters.

In [1]:
import os
import random
import pandas as pd
from tqdm import tqdm
from IPython.display import display, HTML

In [2]:
# variables
use_full_dataset = False
data_dir = '../data' if use_full_dataset else '../data_subsample'

In [3]:
# inputs
fn_metadata = f'{data_dir}/intermediary/element_metadata.jsonl.gz'
fn_trends = '../data/input/trending_searches_by_category.csv'

# outputs
fn_topic = f'{data_dir}/output/tables/topic_area_freq_by_category/'
os.makedirs(fn_topic, exist_ok=True)

In [4]:
df = pd.read_json(fn_metadata, 
                  lines=True)

trends = pd.read_csv(fn_trends)

In [5]:
# create columns that are just the search query.
df.loc[:, 'search'] = df.fn_input.apply(lambda x : ('').join(x.split('/html')[0].split('/')[8:]))
trends.loc[:, 'search'] = trends['search'].str.replace(' ', '-')

In [6]:
trends.search.nunique()

11648

In [7]:
cat2searches = {}
for cat in trends.category.unique():
    searches = (trends[trends.category == cat].search.tolist())
    cat2searches[cat] = searches

In [8]:
cat2desc = {
    'b' : 'Business',
    'e' : 'Entertainment',
    'm' : 'Health',
    't' : 'Science and Tech',
    's' : 'Sports',
    'h' : 'Top Stories',
    'All' : 'Multiple Categories'
}

In [9]:
import random
from scipy.stats import t

In [10]:
cat2df = {}
for k, v in cat2searches.items():
    _df = df[df.search.isin(v)]
    random.seed(303)
    sample = _df.fn_input.unique()
    random.shuffle(sample)
    sample = sample[:700]
    cat2df[k] = _df[_df.fn_input.isin(sample)]

cat2df['All'] = pd.concat(_df for _df in cat2df.values())

In [14]:
label2publabel = {
    'link' : 'Google Product',
    'answer' : 'Google Answer',
    'organic' : 'Non-Google',
    'amp' : 'AMP',
    'ads' : 'Ads',
}

In [15]:
labels = [
    'organic',
    'amp',
    'link',
    'answer',
    'ads'
]

In [16]:
cat2df2 = {}
for cat, _df in cat2df.items():
    print(f"{cat2desc.get(cat, cat)} (N={_df.fn_input.nunique():,})")
    total_area = _df["area_above_the_fold"].sum()
    total_area_fp = _df['area_page'].sum()
    n_pages = _df.fn_input.nunique()
    data = {}
    for label in labels:
        __df = _df[_df.label == label]
        area = (__df["area_above_the_fold"].sum() / total_area)
        area_page = (__df['area_page'].sum() / total_area_fp)
        freq = __df[__df["area_above_the_fold"] != 0].fn_input.nunique() / n_pages
        freq_page = __df[__df['area_page'] != 0].fn_input.nunique() /n_pages
        data[label2publabel.get(label,label)] = {
            ('Top of page', 'area') : area,
            ('Top of page', 'freq') : freq,
            ('First full page', 'area') : area_page,
            ('First full page', 'freq') : freq_page,
        }

    breakdown = pd.DataFrame(data).T
    breakdown.columns = pd.MultiIndex.from_tuples([   
        ('Top of page', 'area'),
        ('Top of page', 'freq'),
        ('First full page', 'area'),
        ('First full page', 'freq')
    ])
    res = (breakdown * 100).round(1).astype(str) + '%'
    display(HTML(res.to_html()))
    
    # save it
    fn_topic_ = fn_topic + f'{cat2desc.get(cat)}.csv'
    breakdown.to_csv(fn_topic_)
    
    cat2df2[cat] = breakdown
    print("*" * 79)

Business (N=700)


*******************************************************************************
Health (N=700)


*******************************************************************************
Entertainment (N=700)


*******************************************************************************
Science and Tech (N=700)


*******************************************************************************
Sports (N=700)


*******************************************************************************
Top Stories (N=700)


*******************************************************************************
Multiple Categories (N=4,200)


*******************************************************************************


In [17]:
res = (breakdown * 100).round(1).astype(str) + '%'

## Deltas
For each topic, we can see how each metric varies from "All".

In [18]:
for cat, _df in cat2df2.items():
    if cat != 'All':
        print(f"{cat2desc.get(cat, cat)}")
        diff = _df - cat2df2['All']
        res = (diff * 100).round(1).astype(str) + '%'
        display(HTML(res.to_html()))
        print("*" * 79)

Business


*******************************************************************************
Health


*******************************************************************************
Entertainment


*******************************************************************************
Science and Tech


*******************************************************************************
Sports


*******************************************************************************
Top Stories


*******************************************************************************
